In [9]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2 
import io

In [19]:
# If you turn this feature on, you can display each 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In-line SQL

In [2]:
%load_ext sql

In [3]:
%%sql postgresql://weaves:3lbpb84@j1/weaves
select count(*) from actor

1 rows affected.


count
200


In [4]:
version = %sql select version()

 * postgresql://weaves:***@j1/weaves
1 rows affected.


In [5]:
version

version
"PostgreSQL 11.5 (Debian 11.5-1+deb10u1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit"


In [7]:
engine = create_engine('postgresql+psycopg2://weaves:3lbpb84@j1:5432/weaves')

In [11]:
tbl0 = 'actor'

In [12]:
df0 = pd.read_sql_query("SELECT * FROM %s;" % tbl0, engine)

In [15]:
df0 = pd.read_sql_table(tbl0, engine)

In [18]:
df0.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [50]:
## Getting tables from a q/kdb+ server
url0="http://j1:4444/q.csv?users2"
c = pd.read_csv(url0)
c.head()

,userid,yob,ctry,dt0,plan,push1,email1,ncontacts,acnd0,age0,nt,tdays
0,0,1989,PL,2018-01-13,STANDARD,1.0,1.0,3,648,30,389,1.665810
1,1,1975,GB,2018-01-29,STANDARD,NaN,NaN,21,632,44,129,4.899225
2,2,1987,PL,2018-01-18,STANDARD,0.0,0.0,21,643,32,96,6.697917
3,3,1994,FR,2018-01-15,STANDARD,1.0,0.0,0,646,25,268,2.410448
4,4,1985,GB,2018-01-11,STANDARD,NaN,NaN,2,650,34,47,13.829790
